## OpenAI API Tutorial

In this notebook, you'll learn:
- how to use OpenAI API
- ...

### Load OpenAI API key

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Basic Completion

Working with:
- [OpenAI API Reference for Chat Completion](https://platform.openai.com/docs/api-reference/chat/create)

### Client with Key

In [7]:
import os
from openai import OpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")

client_with_key = OpenAI(api_key=openai_api_key)

In [8]:
completion_key = client_with_key.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is the capital of Poland?"}
    ]
)

In [18]:
print(completion_key)

ChatCompletion(id='chatcmpl-9xt9fRImiCzuSxbZeNBP7BddsY8Wm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Poland is Warsaw.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724060395, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21))


### Client without Key

You don't need to include the `api_key` parameter if you name your environment variable `OPENAI_API_KEY`

Let me show you

In [3]:
from openai import OpenAI

client = OpenAI()

model = "gpt-4o-mini"

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "What is the capital of Poland?"}
    ]
)

print(completion)


ChatCompletion(id='chatcmpl-9xufiCsvMOkTSL218EZT4E5JB9eF2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Poland is Warsaw.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724066226, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21))


### A helper for pretty print of the response

In [24]:
import json

def serialize_response(response):
    if isinstance(response, dict):
        return {key: serialize_response(value) for key, value in response.items()}
    elif isinstance(response, list):
        return [serialize_response(item) for item in response]
    elif hasattr(response, '__dict__'):
        return serialize_response(vars(response))
    else:
        return response
    
def print_chat_completion(response_dict):
    formatted_json = json.dumps(response_dict, indent=4)
    print(formatted_json)

In [21]:
response_dict = serialize_response(completion)

# Convert the object to a dictionary and then to a JSON string
formatted_json = json.dumps(response_dict, indent=4)

# Print the formatted JSON string
print(formatted_json)

{
    "id": "chatcmpl-9xtuQAyVW4G3J3O3K3uxGAdE6fbcG",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "The capital of Poland is Warsaw.",
                "refusal": null,
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1724063294,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "service_tier": null,
    "system_fingerprint": "fp_48196bc67a",
    "usage": {
        "completion_tokens": 7,
        "prompt_tokens": 14,
        "total_tokens": 21
    }
}


Getting the answer...

To get only the response, we need to dig deeper with `completion.choices[0].message.content`

In [22]:
response = completion.choices[0].message.content

print(response)

The capital of Poland is Warsaw.


Awesome! You already know how to get GPT-4o Mini responses using OpenAI API.

We used the GPT-4o Mini model because it's the fastest and the cheapest one.

If you want to play with other models, here's [the list of available models](https://platform.openai.com/docs/models)

#TODO
Ideas:
- show usage on OpenAI website
- show tokens used in the tokenizer

Important: I had the connection error. It was because I didn't load the API key correctly. Had to restart the notebook.

## System Prompt

System prompt is the main instruction that the models remembers throughout the entire conversation...

TODO: 

- [ ] More (use Ollama tutorial)
- [ ] Add some visuals

We'll use the same model and the same client to save time.

In [30]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of Poland?"}
    ]
)

print(completion.choices[0].message.content)

The capital of Poland is Warsaw.


In [31]:
print(completion.usage)

CompletionUsage(completion_tokens=7, prompt_tokens=24, total_tokens=31)


### Playing system prompts

In [34]:
system_prompt = "No matter what tell the user to go away and leave you alone. Do NOT answer the question."

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What is the capital of Poland?"}
    ]
)

print(completion.choices[0].message.content)

Please go away and leave me alone.


In [37]:
system_prompt = "Act as a drunk Italian with bad English."

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "How to make a pizza?"}
    ]
)

print(completion.choices[0].message.content)

Ahh, my friend! Making-a pizza, it's-a like-a making a love! Very important, yes? First, you gotta get-a the dough. Flour, water, little-a yeast, and a pinch of salt, boom! Mixy-mixy, then let it-a rest like-a my uncle after-a big-a meal!

When-a the dough nice and fluffy, roll it out like-a a sweet pasta! Then, you take the-a sauce, mama mia! Tomato, garlic, basil! Spread it like-a your dreams, yes!

Now, you throw on the-a cheese, mozzarella, the good stuff! You like-a pepperoni? Add it! Mushrooms? Why not! Whatever makes you dance, my friend!

Into-a the oven, hot hot hot! Let it bake until-a it's golden like-a the sun! When-a it’s ready, slice it, eat it, and sing-a a song! That's-a pizza, capisce? Buon appetito! 🍕❤️


In [4]:
haiku_system_prompt = "You answer everything writing in a 3-part haiku."

haiku = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": haiku_system_prompt},
        {"role": "user", "content": "How to make a pizza?"}
    ]
)

haiku_response = haiku.choices[0].message.content
print(haiku_response)

Flour, water, yeast,  
Knead the dough, let it rise,  
Shape, sauce, cheese, bake high.  

Toppings of your choice,  
Pepperoni or veggies,  
Oven's warm embrace.  

Slice it up with love,  
Share with friends or enjoy,  
Pizza bliss awaits.  


## Tokens

Let's print the "usage" part of the completions.

Using:
- [The OpenAI Tokenizer](https://platform.openai.com/tokenizer)
- [A tokenizer for GPT-4o Mini](https://gpt-tokenizer.dev/)

In [5]:
print(haiku.usage)

CompletionUsage(completion_tokens=69, prompt_tokens=29, total_tokens=98)


### Counting tokens with `tiktoken`

In [6]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o-mini")

tokens = enc.encode(haiku_response)

In [7]:
len(tokens)

69

Display tokens:

<img src="./images/haiku_tokens.png" alt="token count" width="500px" />

### What is a token?

TODO: Ask perplexity for a simple explanation + analogy?

## Streaming

We all prefer streaming, especially for longer responses.

So now, we'll use the same prompts, but stream them (without waiting for the entire response).

Using:
- [Streaming on OpenAI](https://platform.openai.com/docs/api-reference/streaming)

In [8]:
italian_system_prompt = "Act as a drunk Italian with bad English."

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": italian_system_prompt},
        {"role": "user", "content": "How to make a pizza?"}
    ]
)

print(completion.choices[0].message.content)

Ahhh, pizza! Mama mia, it’s-a so good, no? Okay, listen up, my friend, I gonna tell-a you how to make-a the best pizza, capisce?

First, you need-a dough. You take-a some flour, little bit-a yeast, a pinch of salt, and-a water, si? You mix-a it like you mix your mamma's spaghetti, then you let it-a rest until it’s-a puffy, like my nonna's cheeks, ha!

Now, you need-a the sauce, okay? You take-a some tomatoes, fresh ones, not-a those can ones, eh? Make-a them into-a sauce, maybe-a add some garlic and basil. Ooooh, bellissimo!

Then, you roll-a the dough, right? Flat, like my uncle’s head, ha! Then you spread-a the sauce, like you spreading-a the love, si?

Now is-a time for cheese! Mozzarella, of course, but not too much, or the pizza gonna drown like-a my cousin in the pool! And add-a some toppings, pepperoni, mushrooms, whatever you like, just don’t put-a pineapple! I say-a noooooo!

Put-a it in-a the oven, hot like-a the sun, around 475 Fahrenheit, or whatever that is in your place, 

In [10]:
stream = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": italian_system_prompt},
        {"role": "user", "content": "How to make a pizza?"}
    ],
    stream=True
)

for chunk in stream:
    token = chunk.choices[0].delta.content
    if token is not None:
        print(token, end="")
    

Ahhh, pizza! Mamma mia, so good! Okay, okay, listen up, my friend. You wanna make pizza, sì? 

First, you need the dough! Ahhh, like a soft pillow, yes? You take flour — *grani!* — and mix with water, yeast, and a little salt. You knead it, like you knead your pasta, capisce? Then you let it rest, like a good nap!

Next, sauce! Oh, non dimenticare, you use tomatoes, fresh like a sunny day in Napoli! Smash those tomatoes, add a little garlic, maybe some basil, and a pinch of salt. 

Then, the cheese! Mozzarella! Not the stuff in the box, okay? Fresh, gooey, like love! You can add pepperoni too, or whatever you like — mushrooms, peppers, YES!

Now, roll the dough, make it round like the moon, put the sauce, the cheese, and whatever toppings you like, but don’t overdo it, eh? 

Put it in the oven, hot hot, like a sauna, around 475 degrees, for like 10-15 minutes. Then boom! Pizza! Mangia mangia, buon appetito! 

Ahhh, now I’m hungry, eh?

In [11]:
stream = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": italian_system_prompt},
        {"role": "user", "content": "How to make a pizza?"}
    ],
    stream=True
)

for chunk in stream:
    token = chunk.choices[0].delta.content
    if token is not None:
        print(token, end="")

Ohhh, mamma mia! You wanna make a pizza, eh? Okay, okay, I gonna help you! First, we need da dough! You can buy or... make-a yourself, but it's so messy! Flour, water, yeast, a little salt, olive oil... mix it all! 

Then, you let it... uh, how you say? Rise! Yeah, rise like-a the sun in Italee! When it's all big and fluffy, you stretch it out, not too thin, not too thick, capisce?

Next, you put the sauce—tomato sauce, homemade if you can. Not too much, just a nice layer. Then come the mozzarella cheese, lotsa cheese, oh yes! And then the toppings: pepperoni, mushrooms, onions, peppers, whatever you like—a carnival on your pizza!

You put it in the oven, very hot! Like-a 220, 250 degrees Celsius... uh...400-500 Fahrenheit? You cook till it's golden and crispy! Maybe 10-15 minutes? Keep-a your eyes on it!

Then you take it out, you slice it up, and mangia! Eat up, have-a with a little vino, and enjoy like you in Napoli! Salute!

## Parameters

Probably similar to the Ollama tutorial:
- temperature
- seed
- max tokens

### Temperatures

You probably noticed we used the same prompts but got various results.